#Connect Gdrive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#Pre trained models

In [ ]:
import os
if not os.path.exists("/content/drive/MyDrive/sovits4data/logs/44k"):
  os.makedirs("/content/drive/MyDrive/sovits4data/logs/44k")

!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/D_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k
!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/G_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k

#Cloning repo and installing dependencies

In [ ]:
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
!pip uninstall -y torchdata torchtext
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq tensorboardX torchcrepe librosa==0.9.1
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

%cd /content/so-vits-svc

!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain

!curl -L https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr -o /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt

Cloning into 'so-vits-svc'...
remote: Enumerating objects: 1889, done.
remote: Counting objects: 100% (669/669), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 1889 (delta 579), reused 579 (delta 546), pack-reused 1220
Receiving objects: 100% (1889/1889), 9.14 MiB | 27.29 MiB/s, done.
Resolving deltas: 100% (1088/1088), done.
Found existing installation: torchdata 0.6.1
Uninstalling torchdata-0.6.1:
  Successfully uninstalled torchdata-0.6.1
Found existing installation: torchtext 0.15.2
Uninstalling torchtext-0.15.2:
  Successfully uninstalled torchtext-0.15.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 16.4 MB/s e

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 86.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

#Writing. files and paths

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"

RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"
config_path = "configs/config.json"
sync_raw_and_results = True
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results


sync_configs_and_logs = True
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

#Link dataset

In [ ]:
# link dataset in dataset_raw
!ln -s /content/drive/MyDrive/sliced/ /content/so-vits-svc/dataset_raw/

#Resample

In [ ]:
%cd /content/so-vits-svc
!python resample.py

/content/so-vits-svc
./dataset_raw/sliced
133it [00:25,  5.12it/s]


#Generate config

In [ ]:

%cd /content/so-vits-svc

# speech_encoder = "vec768l12"

# from pretrain.meta import download_dict
# download_dict = download_dict()

# url = download_dict[speech_encoder]["url"]
# output = download_dict[speech_encoder]["output"]

# import os
# if not os.path.exists(output):
#   !curl -L {url} -o {output}

!python preprocess_flist_config.py #--speech_encoder={speech_encoder}

/content/so-vits-svc
100% 1/1 [00:00<00:00, 16.85it/s]
Writing ./filelists/train.txt
100% 1056/1056 [00:00<00:00, 1370840.30it/s]
Writing ./filelists/val.txt
100% 2/2 [00:00<00:00, 64527.75it/s]
Writing configs/config.json


#Generate hubert

In [ ]:
%cd /content/so-vits-svc
!python preprocess_hubert_f0.py

/content/so-vits-svc
[1058]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1

#Saving

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"

!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"

  adding: content/so-vits-svc/dataset/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/sliced/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_32_3_600.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_0_5_22.wav.f0.npy (deflated 41%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_10_4_225.wav.f0.npy (deflated 54%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_22_24_476.wav.f0.npy (deflated 44%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_live_82_100.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_live_90_109.wav (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_28_7_549.wav (deflated 6%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_6_0_137.spec.pt (deflated 15%)
  adding: content/so-vits-svc/dataset/44k/sliced/segments_vocal_41_

# loading preprocessed

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

cp: '/content/drive/MyDrive/sovits4data/configs/' and '/content/so-vits-svc/configs' are the same file
cp: '/content/drive/MyDrive/sovits4data/filelists/' and '/content/so-vits-svc/filelists' are the same file
Archive:  /content/drive/MyDrive/sovits4data/dataset.zip
   creating: /content/so-vits-svc/dataset/
   creating: /content/so-vits-svc/dataset/44k/
   creating: /content/so-vits-svc/dataset/44k/sliced/
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_vocal_32_3_600.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_vocal_0_5_22.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_vocal_10_4_225.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_vocal_22_24_476.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_live_82_100.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments_live_90_109.wav  
  inflating: /content/so-vits-svc/dataset/44k/sliced/segments

#Train

In [ ]:
tensorboard_on = False

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k


%cd /content/so-vits-svc
!python train.py -c configs/config.json -m 44k

/content/so-vits-svc
INFO:44k:{'train': {'log_interval': 200, 'eval_interval': 800, 'seed': 1234, 'epochs': 10000, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 6, 'fp16_run': False, 'lr_decay': 0.999875, 'segment_size': 10240, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0, 'use_sr': True, 'max_speclen': 512, 'port': '8001', 'keep_ckpts': 1, 'all_in_mem': False}, 'data': {'training_files': 'filelists/train.txt', 'validation_files': 'filelists/val.txt', 'max_wav_value': 32768.0, 'sampling_rate': 44100, 'filter_length': 2048, 'hop_length': 512, 'win_length': 2048, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': 22050}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2, 2], 'upsample_initial_channel':

In [ ]:
%cd /content/so-vits-svc
!python cluster/train_cluster.py

#Infer

In [ ]:
wav_filename = "cheque.wav"
model_filename = "G_128800.pth"
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "sliced"
trans = "0"
cluster_infer_ratio = "0"
auto_predict_f0 = False
apf = ""
if auto_predict_f0:
  apf = " -a "
f0_mean_pooling = False
fmp = ""
if f0_mean_pooling:
  fmp = " -fmp "

enhance = False
ehc = ""
if enhance:
  ehc = " -eh "

config_filename = "config.json"
config_path = "/content/so-vits-svc/configs/" + config_filename
kmeans_filenname = "kmeans_10000.pt"
kmeans_path = "/content/so-vits-svc/logs/44k/" + kmeans_filenname
slice_db = "-40"
wav_format = "flac"
wav_output = "/content/so-vits-svc/results/" + wav_filename + "_" + trans + "key" + "_" + speaker + "." + wav_format

%cd /content/so-vits-svc
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} {fmp} {ehc}

download_after_inference = True

if download_after_inference:
  from google.colab import files
  files.download(wav_output)

/content/so-vits-svc
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_features': 0.1, 'final_dim

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>